In [1]:
import plotly
import pandas as pd
import numpy as np

plotly.offline.init_notebook_mode(connected=True)

In [8]:
df = pd.read_csv('city_info_geocodio_2.csv')
df.shape
df.head(3)

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City.1,State.1,County,Zip,Country,Source,Congressional District
0,Aurora,Illinois,41.747452,-88.238622,1.0,place,NaN,NaN,Aurora,IL,DuPage County,60504.0,US,TIGER/Line® dataset from the US Census Bureau,IL11
1,Carbondale,Illinois,37.739004,-89.209109,1.0,place,NaN,NaN,Carbondale,IL,Jackson County,62901.0,US,TIGER/Line® dataset from the US Census Bureau,IL12
2,Champaign,Illinois,40.107890,-88.244266,1.0,place,NaN,NaN,Champaign,IL,Champaign County,61820.0,US,TIGER/Line® dataset from the US Census Bureau,IL13


In [9]:
# Geocodio told me that five values weren't succesfful mapped, but for the sake of best practice, I checked anyway
# I suspect that these either weren't named correctly or for some reason weren't in geocodio's database 
# Because there are only five missing values, I'll fix this manually, although that obviously isn't a scalable 
# solution. 

df.loc[df['Latitude'] == 0]

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City.1,State.1,County,Zip,Country,Source,Congressional District
63,Blue River,Colorado,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,Rangley,Colorado,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Polk County,Florida,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,Bexley,Ohio,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,Hamilton County Parks,Ohio,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Geocodio tells you the likely accuracy of the lat/long it's provided with an accuracy score. Some of the scores do not have high accuracy scores, which is worth mentioning. For now, we'll see if any mapping errors actually occur. If they do, I'll go back and engineer a solution.

In [10]:
#We have 239 cities, so we want to make sure there's 239 rows. 
df.shape

(239, 15)

Because I'm only missing five values, I'm going to manually find their lat/lon coordinates and then add them in. I'll do this here because I want them mapped in all future data. 

In [11]:
# Manual updating of relevent values 

new_data = [(40.0874759, -108.8048292, 'CO3', 'CO'), (39.446649, -106.03757, 'CO2', 'CO'), 
            (28.022243, -81.732857, 'FL9', 'FL'), (39.9689532, -82.9376804, 'OH3', 'OH'), 
            (39.103119, -84.512016, 'OH1', 'OH')]

indexes = (63, 85, 98, 172, 191)

for i in range(5): 
    df.loc[indexes[i],'Latitude'] = new_data[i][0]
    df.loc[indexes[i], 'Longitude'] = new_data[i][1]
    df.loc[indexes[i], 'Congressional District'] = new_data[i][2]
    df.loc[indexes[i], 'State.1'] = new_data[i][3]


In [12]:
# Checking to see we've fixed all lat/lon problems 

df.loc[df['Latitude'] == 0]

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,Number,Street,City.1,State.1,County,Zip,Country,Source,Congressional District


In [13]:
# Dropping unnecessary columns added by Geocodio for the sake of data tidiness 

df = df.drop(columns = ['Number', 'Street', 'City.1', 'Source'])
df.head(1)

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District
0,Aurora,Illinois,41.747452,-88.238622,1.0,place,IL,DuPage County,60504.0,US,IL11


Much better. The campaign wants to keep track of the status of the anti-panhandling statutes/ordinances in each city, and they want to be able to update those values to reflect varying degrees of success as the campaign goes on. I'm going to create a 'status' value for each city set it's default to "active." Then I'm going to map "status text" and "marker color" right onto the values.

In [14]:
# Adding in a "status" column

df['status'] = 0


# Creating a conditional column explaining ordinance status

d_text = {0: 'Ordinance Active - With No Response', 
          1: 'Ordinance Active - With Response Indicating No Immediate Repeal',
          2: 'Ordinance Active - With Committment To Review', 
          3: 'Ordinance Halted - With Committment to Review',
          4: 'Ordinance Repealed'}

df['statusText'] = df['status'].map(d_text)

# Setting point color conditionally based upon status 

d_color = {0: 'rgb(255, 0, 0)',
           1: 'rgb(255, 192, 203)',
           2: 'rgb(255, 165, 0)', 
           3: 'rgb(255, 255, 0)', 
           4: 'rgb(127, 255, 0)'}

df['color'] = df['status'].map(d_color)

In [15]:
# print(df.dtypes)
df.head(3)

,City,State,Latitude,Longitude,Accuracy Score,Accuracy Type,State.1,County,Zip,Country,Congressional District,status,statusText,color
0,Aurora,Illinois,41.747452,-88.238622,1.0,place,IL,DuPage County,60504.0,US,IL11,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"
1,Carbondale,Illinois,37.739004,-89.209109,1.0,place,IL,Jackson County,62901.0,US,IL12,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"
2,Champaign,Illinois,40.107890,-88.244266,1.0,place,IL,Champaign County,61820.0,US,IL13,0,Ordinance Active - With No Response,"rgb(255, 0, 0)"


Now I'm going to persist my changes. I'll make updates periodically, but I'll use a seperate notebook for that so that I don't have to continously repeat this project. 

In [17]:
df.to_csv('cleaned_data.csv', mode ='w+')